# Explore here

It's recommended to use this notebook for exploration purposes.

In [ ]:
import requests
import time

# Descargar HTML de la página web
link = " https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
data = requests.get(link, time.sleep(10)).text

data

In [ ]:
# Paso 3 Transforma el HTML

from bs4 import BeautifulSoup
import pandas as pd

# Parsear el HTML usando BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')
soup


In [ ]:
#Find all tables
tables = soup.find_all("table")
tables

In [ ]:
import pandas as pd

for index, table in enumerate(tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        table_index = index
        break

# Crear un DataFrame
tesla_revenue = pd.DataFrame(columns = ["Date", "Revenue"])
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Date = col[0].text
        Revenue = col[1].text.replace("$", "").replace(",", "")
        tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({
            "Date": Date,
            "Revenue": Revenue
        }, index = [0])], ignore_index = True)

tesla_revenue.head()

In [ ]:
# Procesamos el Data frame
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]
tesla_revenue

In [ ]:
# Almacenar datos en SQLITE 
import sqlite3

connection = sqlite3.connect("Tesla.db")
connection

In [ ]:
# Creamos tabla

cursor = connection.cursor()
cursor.execute("""CREATE TABLE revenue (Date, Revenue)""")

In [ ]:
tesla_tuples = list(tesla_revenue.to_records(index = False))
tesla_tuples[:5]

In [ ]:
# Comprueba los datos del database
for row in cursor.execute("SELECT * FROM revenue"):
    print(row)

GRÁFICOS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axis = plt.subplots(figsize = (10, 5))

tesla_revenue["Date"] = pd.to_datetime(tesla_revenue["Date"])
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].astype('int')
sns.lineplot(data = tesla_revenue, x = "Date", y = "Revenue")

plt.tight_layout()

plt.show()

In [ ]:
fig, axis = plt.subplots(figsize = (10, 5))

tesla_revenue["Date"] = pd.to_datetime(tesla_revenue["Date"])
tesla_revenue_yearly = tesla_revenue.groupby(tesla_revenue["Date"].dt.year).sum().reset_index()

sns.barplot(data = tesla_revenue_yearly[tesla_revenue_yearly["Date"] < 2023], x = "Date", y = "Revenue")

plt.tight_layout()

plt.show()

In [ ]:
fig, axis = plt.subplots(figsize = (10, 5))

tesla_revenue_monthly = tesla_revenue.groupby(tesla_revenue["Date"].dt.month).sum().reset_index()

sns.barplot(data = tesla_revenue_monthly, x = "Date", y = "Revenue")

plt.tight_layout()

plt.show()